In [1]:
import import_ipynb
import functions as f

importing Jupyter notebook from functions.ipynb


In [2]:
# y^2 = x^3 + x + 3 over F_43
p = 43
c_4 = 1

In [3]:
P = (19, 42)
P_A = (28, 15)

In [4]:
f.DLP_BSGS(P, P_A, c_4, p, True)

0P = None
1P = (19, 42)
2P = (36, 13)
3P = (29, 13)
4P = (9, 15)
5P = (21, 30)
a_0 = 3

P_A - 0mP = (28, 15)
P_A - 1mP = (29, 13)
a_1 = 1

a = 9
1P = (19, 42)
2P = (36, 13)
3P = (29, 13)
4P = (9, 15)
5P = (21, 30)
6P = (39, 35)
7P = (2, 23)
8P = (17, 26)
9P = (28, 15)

P_A = (28, 15)


9